In [2]:
import psycopg2
from tqdm import tqdm

# Database connection
connection = psycopg2.connect(
    host='128.1.227.35',
    user='postgres',
    password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    database='medols'
)

try:
    with connection.cursor() as cursor:
        # Find duplicate instagram_post_id
        find_duplicates_sql = """
        SELECT instagram_post_id, COUNT(*)
        FROM instagram_post_sentiment
        GROUP BY instagram_post_id
        HAVING COUNT(*) > 1
        """
        cursor.execute(find_duplicates_sql)
        duplicates = cursor.fetchall()

        # Handle duplicates with tqdm progress bar
        for instagram_post_id, count in tqdm(duplicates, desc="Processing duplicates"):
            # print(f"Found {count} duplicates for instagram_post_id: {instagram_post_id}")

            # Select rows with the duplicate instagram_post_id
            select_duplicates_sql = """
            SELECT id
            FROM instagram_post_sentiment
            WHERE instagram_post_id = %s
            """
            cursor.execute(select_duplicates_sql, (instagram_post_id,))
            duplicate_rows = cursor.fetchall()

            # Keep the first occurrence and remove the others
            ids_to_delete = [row[0] for row in duplicate_rows[1:]]

            if ids_to_delete:
                delete_duplicates_sql = """
                DELETE FROM instagram_post_sentiment
                WHERE id IN (%s)
                """ % ', '.join(map(str, ids_to_delete))
                cursor.execute(delete_duplicates_sql)
                # print(f"Deleted {len(ids_to_delete)} duplicate rows for instagram_post_id: {instagram_post_id}")

        # Commit the changes
        connection.commit()

finally:
    connection.close()

# Now add the unique constraint
connection = psycopg2.connect(
    host='128.1.227.35',
    user='postgres',
    password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    database='medols'
)

try:
    with connection.cursor() as cursor:
        add_constraint_sql = """
        ALTER TABLE instagram_post_sentiment
        ADD CONSTRAINT unique_instagram_post_id UNIQUE (instagram_post_id)
        """
        cursor.execute(add_constraint_sql)
        connection.commit()
        print("Unique constraint added to instagram_post_id")

finally:
    connection.close()


Processing duplicates: 100%|█| 1053/1053 [00


Unique constraint added to instagram_post_id
